<a href="https://colab.research.google.com/github/Mirabila/MyApi/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
    #    print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Prepare Data for YOLOv5 Format

**Create directories to hold images and txt files for annotations (train , val ,test)**

**classes**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**calculate ancor points and hiegt width of bonding boxes**

# split train val test

In [ ]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('/content/drive/MyDrive/kaggle/kaggle/input/hard-hat-detection/images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

total = 5000
train : 4000
val   : 500
test  : 500


# Get YOLOv5 model

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12344, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 12344 (delta 21), reused 29 (delta 15), pack-reused 12292
Receiving objects: 100% (12344/12344), 12.73 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (8492/8492), done.
/content/yolov5


**create yaml file for configuration**

In [ ]:
import yaml

dict_file = {'train':'/content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/train' ,
            'val': '/content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/val',
            'nc' : '3',
            'names' : ['helmet','head','person']}

with open('/content/yolov5/data/hard_head.yaml', 'w+') as file:
    documents = yaml.dump(dict_file, file)

# Train Net

In [ ]:
!python train.py --img 640 --batch 16 --epochs 20 --data '/content/yolov5/data/hard_head.yaml' --cfg models/yolov5s.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=/content/yolov5/data/hard_head.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: YOLOv5 requirements "ipython" "thop>=0.1.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 2 packages updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun 

# Detect (predict)

In [ ]:
#!python detect.py --source /kaggle/working/Dataset/images/test --img-size 416 --conf 0.4 --weights weights.pt 
!python detect.py --source /content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/test --data data/hard_head.yaml  --weights /content/yolov5/runs/train/exp/weights/best.pt --conf 0.25

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/test, data=data/hard_head.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-117-ga1e5f9a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/500 /content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/test/hard_hat_workers1004.png: 640x640 8 helmets, 1 person, 12.7ms
image 2/500 /content/drive/MyDrive/kaggle/kaggle/working/Dataset/images/test/hard_hat_workers1012.png: 640x640 5 helmets, 12.7ms
image 3/500 /

In [ ]:
!cp "/content/yolov5" -r "/content/drive/MyDrive/yolov5"